In [1]:
import h2o
import os

#start the h2o cluster
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.


H2OServerError: Cluster reports unhealthy status

In [ ]:
#Check if file exists
if not os.path.exists("./data_input/LoanStats3a.csv"):
    input_csv = "https://s3-us-west-2.amazonaws.com/h2o-tutorials/data/topics/lending/lending_club/LoanStats3a.csv"

loans = h2o.import_file(input_csv,
                        col_types = {
                            "int_rate":"string",
                            "revol_util":"string",
                            "emp_length":"string",
                            "verification_status":"string"
                        })

In [ ]:
num_unfiltered_loans = loans.dim[0]
loans["loan_status"].table().head(15)

In [ ]:
#Filtering loans, remove those in process

ongoing_status = ["Current",
                  "In Grace period", 
                  "Later (16-30 days)",
                  "Late (31-120 days)",
                  "Does not meet the credit policy. Status:Current",
                  "Does not meet the credit policy. Status:In Grace Period"]

loans = loans[~loans["loan_status"].isin(ongoing_status)]
num_filtered_loans = loans.dim[0]
loans["loan_status"].table().head(20)


In [4]:

h2o.cluster().shutdown(prompt = True)

AttributeError: 'NoneType' object has no attribute 'shutdown'